In [8]:
import numpy as np
import pandas as pd
from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian
from nilearn.datasets import fetch_atlas_schaefer_2018
from netneurotools.datasets import fetch_schaefer2018

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [157]:

# define path
path = '/Users/laurituominen/Documents/Research/Reettis/neuromaps/'

# define parcellations 
schaefer_vol = fetch_atlas_schaefer_2018(n_rois=200, yeo_networks=7)  # can change 200 to a different number of parcels
parcellation_mni = schaefer_vol['maps']
parcellater_mni = Parcellater(schaefer_vol['maps'], 'mni152')

schaefer_surf = fetch_schaefer2018(version='fsaverage')['200Parcels7Networks']  # can change 200 to a different number of parcels
parcellation_fs164k =annot_to_gifti(schaefer_surf) 
parcellater_fs164k = Parcellater(annot_to_gifti(schaefer_surf), 'fsaverage')

schaefer_surf10 = fetch_schaefer2018(version='fsaverage5')['200Parcels7Networks']  # can change 200 to a different number of parcels
parcellation_fs10k =annot_to_gifti(schaefer_surf10) 
parcellater_fs10k = Parcellater(annot_to_gifti(schaefer_surf10), 'fsaverage')


In [158]:
## load turku data
img_L = load_data(path + 'data/lh.sig.nii')
img_gii_L = construct_shape_gii(img_L)

img_R = load_data(path + 'data/rh.sig.nii')  
img_gii_R = construct_shape_gii(img_R)

#parcellate the turku_map into turku_parc
turku_map = (img_gii_L, img_gii_R)
turku_parc = parcellater_fs164k.fit_transform(turku_map, space='fsaverage', ignore_background_data=True)

In [159]:
# get annotations 
annotations = list(fetch_annotation(source=['hcps1200',
                                            'margulies2016',
                                            'raichle',
                                            'ding2010', 
                                            'finnema2016', 
                                            'dubois2015',
                                            'gallezot2010',
                                            'gallezot2017',
                                            'hillmer2016',
                                            'jaworska2020',
                                            'kaller2017',
                                            'kantonen2020',
                                            'laurikainen2018',
                                            'normandin2015',
                                            'radnakrishnan2018',
                                            'sandiego2015',
                                            'satterthwaite2014',
                                            'sasaki2012',
                                            'savli2012',
                                            'satterthwaite2014',
                                            'smith2017',
                                            'tuominen',
                                            'mueller2013',
                                            'naganawa2020',
                                            'fazio2016']).keys())

annotations.extend(fetch_annotation(source=['norgaard2021', 'beliveau2017'], space='fsaverage').keys())


In [180]:
# parcellate annotations

# initialize
parcellated = dict([])
for (src, desc, space, den) in annotations:

    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    
    if space == 'MNI152':
        parcellater = parcellater_mni
    elif space == 'fsaverage' and den == '164k':
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den == '164k':
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='164k')
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den != '164k':
        # unfortunately for fsLR-4k we are upsampling to fsaverage-10k to parcellate but it should be fine
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='10k')
        parcellater = parcellater_fs10k
    
    parcellated[desc] = parcellater.fit_transform(annot, space=space, ignore_background_data=True)
    
    if parcellated[desc].shape == (200,):
        parcellated[desc] = parcellated[desc].reshape(1,200)

# error message related to zero division in occipital cortex of carfentanil maps 

/Users/laurituominen/anaconda3/lib/python3.10/site-packages/scipy/ndimage/_measurements.py:803: RuntimeWarning: invalid value encountered in divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)


In [181]:
# precalculated spins for the schaefer200 
spins = np.load(path + 'parcellations/spins_hungarian_schaefer200-7networks.npy')
nspins = spins.shape[1]


In [182]:
nulls_turku = dict([])
corrs_turku = dict([])

# go over annotations 
for src, desc, space, den in annotations:
    if space == 'MNI152':
        parcellation=schaefer_vol['maps']
        
    elif space == 'fsaverage' and den == '164k':
        parcellation=annot_to_gifti(schaefer_surf)
        
    elif space == 'fsLR' and den == '164k':
        parcellation=annot_to_gifti(schaefer_surf)
        
    elif space == 'fsLR' and den != '164k':
        parcellation=annot_to_gifti(schaefer_surf10)
    
    # correlation between annotations and enigma map
    print(desc)
    rho_turku = pearsonr(parcellated[desc][0], turku_parc)[0]
    
    # get 10k rotations 
    rotated = hungarian(data=parcellated[desc], n_perm=nspins, spins=spins, parcellation=schaefer_vol['maps']) 
    
    # get null Turku
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(turku_parc, rotated[:,i])[0]    
    
    # get p-value Turku
    pspin = (1 + sum(abs(n) > abs(rho_turku ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive  
    corrs_turku[src+'_'+desc] = ( (-1 * rho_turku, pspin ) )
    nulls_turku[src+'_'+desc] = n
    
# save correlations & nulls 
np.savez(path + 'data/corrs_turku_schaefer.npz', **corrs_turku)
np.savez(path + 'data/nulls_turku_schaefer.npz', **nulls_turku)


mrb
abp688
madam
ucbj
p943
gsk189254
megalpha
megbeta
megdelta
meggamma1
meggamma2
megtheta
megtimescale
myelinmap
thickness
flubatine
fallypride
sch23390
carfentanil
fmpepd2
fcgradient01
fcgradient02
fcgradient03
fcgradient04
fcgradient05
fcgradient06
fcgradient07
fcgradient08
fcgradient09
fcgradient10
intersubjvar
lsn3172176
omar
gsk215083
cbf
cbv
cmr02
cmruglu
flb457
fepe2i
meancbf
altanserin
dasb
p943
way100635
flb457
feobv
az10419369
cimbi36
cumi101
dasb
sb207145
flumazenil
